# Framework for testing features

In [15]:
#IMPORTS

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from geopy.distance import geodesic
from sklearn.preprocessing import OneHotEncoder

In [16]:
ais_train_data_path = '../../Project materials/ais_train.csv'
ais_test_data_path = '../../Project materials/ais_test.csv'
ports_data_path = '../../Project materials/ports.csv'
vessels_data_path = '../../Project materials/vessels.csv'
schedules_data_path = '../../Project materials/schedules_to_may_2024.csv'



ais_data_train = pd.read_csv(ais_train_data_path, sep='|')
ais_data_test = pd.read_csv(ais_test_data_path, sep=',')
ports = pd.read_csv(ports_data_path, sep='|')
vessels = pd.read_csv(vessels_data_path, sep='|')
schedules = pd.read_csv(schedules_data_path, sep='|')

In [17]:
#FILTER OUT ALL SHIPS WITH LESS THAN 300 ENTRIES FROM THE TRAINING DATAFRAME:

timesteps_per_ship = ais_data_train.groupby('vesselId').size()
vessels_with_enough_timesteps = timesteps_per_ship[timesteps_per_ship >= 300].index

# Step 3: Filter the main dataframe to keep only ships with 300 or more timesteps
ais_data_train = ais_data_train[ais_data_train['vesselId'].isin(vessels_with_enough_timesteps)]


### Create or extract features that are to be tested:

##### Movement status

In [18]:
# EXTRACT MOVEMENT STATUS FROM NAVSTAT:

def categorize_navstat_contrast(navstat):
    if navstat in [0, 8]:
        return 1  # Underway
    elif navstat in [2, 3, 4]:
        return 0.5  # Restricted Movement
    elif navstat in [1, 5, 6]:
        return -1  # Stationary
    else:
        return 0  # Unknown

ais_data_train['movement_status'] = ais_data_train['navstat'].apply(categorize_navstat_contrast)


##### Estimated time to destination

In [19]:
#Preprocess ETA RAW and time

current_year = 2024
ais_data_train['ETARAW_transformed'] = ais_data_train['etaRaw'].apply(lambda x: f"{current_year}-{x}")
ais_data_train['ETARAW_transformed'] = pd.to_datetime(ais_data_train['ETARAW_transformed'], format='%Y-%m-%d %H:%M', errors='coerce')
ais_data_train['time'] = pd.to_datetime(ais_data_train['time'])

In [20]:
# Use linear interpolation to edit nan values

ais_data_train =ais_data_train.sort_values(by=['vesselId', 'time']).reset_index(drop=True)
grouped =ais_data_train.groupby('vesselId')
ais_data_train['ETARAW_transformed'] = grouped['ETARAW_transformed'].apply(lambda x: x.interpolate(method='linear')).reset_index(level=0, drop=True)

ais_data_train['estimated_time_to_destination'] = (ais_data_train['ETARAW_transformed'] -ais_data_train['time']).dt.total_seconds() / 3600

##### Time difference

In [21]:
ais_data_train['time_difference'] = ais_data_train.groupby('vesselId')['time'].diff().dt.total_seconds() / 60
ais_data_train['time_difference'] = ais_data_train['time_difference'].fillna(0)


### Split the dataset into testing and training sets

In [22]:
def split_ship_dataset(ship_dataset):

    last_timestep = ship_dataset['time'].max()
    five_days_ago = last_timestep - pd.Timedelta(days=5)

    test_data = ship_dataset[ship_dataset['time'] > five_days_ago]
    train_data = ship_dataset[ship_dataset['time'] <= five_days_ago]

    return train_data, test_data

In [23]:
ais_data_train = ais_data_train.sort_values(by=['vesselId', 'time'])

train_list = []
test_list = []

for vessel_id, group in ais_data_train.groupby('vesselId'):
    train_data, test_data = split_ship_dataset(group)
    train_list.append(train_data)
    test_list.append(test_data)

train_df = pd.concat(train_list)
test_df = pd.concat(test_list)


In [24]:
print(train_df.shape)
print(test_df.shape)

(1435417, 15)
(85622, 15)


## Testing the one model per ship approach (Random Forest)

Try:
- Implement a random forest classifier for the movement status feature (Transform features between the numeric values to the labels)

In [31]:
baseline_features = ['longitude', 'latitude', 'time_difference', ]
new_features = ['longitude', 'latitude', 'time_difference', 'cog', 'sog', 'rot']

unique_vessel_ids = ais_data_test['vesselId'].unique()


#Hyperparameters
sequence_length = 5
number_of_estimatiors = 50
random_state = 42

In [32]:
baseline_geodesic_distances = []
new_geodesic_distances = []

movement_status_mismatch_count = 0  
total_movement_status_count = 0

In [33]:

for vessel_id in unique_vessel_ids:

    # EXTRACT DATA FOR THE VESSEL AT HAND:
    vessel_df_train = train_df[train_df['vesselId'] == vessel_id]
    vessel_df_test = test_df[test_df['vesselId'] == vessel_id]

    vessel_df_train = vessel_df_train.sort_values(by='time')
    vessel_df_test = vessel_df_test.sort_values(by='time')
    vessel_df_test = vessel_df_test.reset_index(drop=True)

    if len(vessel_df_train) < sequence_length:
        print(f'Not enough historical data to predict for vessel_id: {vessel_id}')
        continue

    total_baseline_features = vessel_df_train[baseline_features].values
    total_new_features = vessel_df_train[new_features].values  

    # CREATE TIMESERIES:
    X_baseline, y_baseline, X_new, y_new, X_mapped_new = [], [], [], [], []
    for i in range(len(total_baseline_features) - sequence_length):
        X_baseline.append(total_baseline_features[i:i + sequence_length])
        X_new.append(total_new_features[i:i + sequence_length])

        y_baseline.append(total_baseline_features[i + sequence_length])
        y_new.append(total_new_features[i + sequence_length])

    X_baseline = np.array(X_baseline)
    X_new = np.array(X_new)
    X_mapped_new = np.array(X_mapped_new)  
    y_baseline = np.array(y_baseline)
    y_new = np.array(y_new)

    # DEFINE MODELS:
    lat_baseline_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)
    lon_baseline_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)
    lat_new_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)
    lon_new_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)
    cog_new_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)
    sog_new_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)
    rot_new_model = RandomForestRegressor(n_estimators=number_of_estimatiors, random_state=random_state)

    #mov_stat_classifier = RandomForestClassifier(n_estimators=number_of_estimatiors, random_state=random_state)

    # TRAIN MODELS:
    lat_baseline_model.fit(X_baseline.reshape(X_baseline.shape[0], -1), y_baseline[:, 1])
    lon_baseline_model.fit(X_baseline.reshape(X_baseline.shape[0], -1), y_baseline[:, 0])
    lat_new_model.fit(X_new.reshape(X_new.shape[0], -1), y_new[:, 1])
    lon_new_model.fit(X_new.reshape(X_new.shape[0], -1), y_new[:, 0])
    cog_new_model.fit(X_new.reshape(X_new.shape[0], -1), y_new[:, 3])
    sog_new_model.fit(X_new.reshape(X_new.shape[0], -1), y_new[:, 4])
    rot_new_model.fit(X_new.reshape(X_new.shape[0], -1), y_new[:, 5])

    #y_mov_stat_class = np.vectorize(movement_status_mapping.get)(y_new[:, 3])
    #mov_stat_classifier.fit(X_mapped_new.reshape(X_mapped_new.shape[0], -1), y_mov_stat_class)

    # PREPARE FOR EVALUATION OF MODELS:
    last_known_sequence_baseline = total_baseline_features[-sequence_length:]
    last_known_sequence_new = total_new_features[-sequence_length:] 

    current_input_baseline = last_known_sequence_baseline.reshape(1, -1)
    current_input_new = last_known_sequence_new.reshape(1, -1)

    true_latitudes = vessel_df_test[baseline_features].iloc[:, 1]
    true_longitudes = vessel_df_test[baseline_features].iloc[:, 0]

    for step in range(len(vessel_df_test)):

        # PREDICTIONS:
        predicted_lat_baseline = lat_baseline_model.predict(current_input_baseline)[0]
        predicted_lon_baseline = lon_baseline_model.predict(current_input_baseline)[0]
        predicted_lat_new = lat_new_model.predict(current_input_new)[0]
        predicted_lon_new = lon_new_model.predict(current_input_new)[0]
        predicted_cog_new = cog_new_model.predict(current_input_new)[0]
        predicted_sog_new = sog_new_model.predict(current_input_new)[0]
        predicted_rot_new = rot_new_model.predict(current_input_new)[0]

        #predicted_mov_stat_category = mov_stat_classifier.predict(current_input_mapped_new.reshape(1, -1))[0]

        #predicted_mov_stat_new = inverse_movement_status_mapping[predicted_mov_stat_category]

        # EVALUATION:
        true_lat = true_latitudes[step]
        true_lon = true_longitudes[step]

        actual_coords = (true_lat, true_lon)
        baseline_predicted_coords = (predicted_lat_baseline, predicted_lon_baseline)

        geodesic_dist_baseline = geodesic(actual_coords, baseline_predicted_coords).kilometers
        baseline_geodesic_distances.append(geodesic_dist_baseline)

        new_predicted_coords = (predicted_lat_new, predicted_lon_new)
        geodesic_dist_new = geodesic(actual_coords, new_predicted_coords).kilometers
        new_geodesic_distances.append(geodesic_dist_new)


        # FIND NEXT INPUTS:
        next_input_baseline = np.array([predicted_lon_baseline, predicted_lat_baseline, 20])
        current_input_baseline = np.hstack([current_input_baseline[:, 3:], next_input_baseline.reshape(1, -1)])

        next_input_new = np.array([predicted_lon_new, predicted_lat_new, 20, predicted_cog_new, predicted_sog_new, predicted_rot_new])  
        
        current_input_new = np.hstack([current_input_new[:, 6:], next_input_new.reshape(1, -1)])
        
    print("Done with a ship")

average_geodesic_baseline = np.mean(baseline_geodesic_distances)
average_geodesic_new = np.mean(new_geodesic_distances)

# VISUALIZE RESULTS
print(f"Baseline Model - Average Geodesic Distance (km): {average_geodesic_baseline}")
print(f"New Model - Average Geodesic Distance (km): {average_geodesic_new}")


Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a ship
Done with a sh